In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

Are we have info about test customers in preiouses data?

In [81]:
import gc
import sys
sys.path.append('../')

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
from IPython.display import HTML, display
from sklearn import metrics

MERED = "merged"
#PATH = "../../../data/home-credit-default-risk/"
PATH = "../../data/home-credit-default-risk/"

In [3]:
ls {PATH}

application_test.csv         HomeCredit_columns_description.csv
application_test.csv.zip     installments_payments.csv
application_train.csv        installments_payments.csv.zip
application_train.csv.zip    merged/
bureau_balance.csv           POS_CASH_balance.csv
bureau_balance.csv.zip       POS_CASH_balance.csv.zip
bureau.csv                   previous_application.csv
bureau.csv.zip               previous_application.csv.zip
credit_card_balance.csv      sample_submission.csv.zip
credit_card_balance.csv.zip


In [4]:
file_names = ['application_train', 'application_test']
tables = [pd.read_csv(f'{PATH}{file_name}.csv',low_memory=False) for file_name in file_names]

In [7]:
for t in tables: display(t.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
for t in tables: display(DataFrameSummary(t).summary())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511,307511,NaN,NaN,NaN,NaN,307511,307511,307511,307499,...,307511,307511,307511,307511,265992,265992,265992,265992,265992,265992
mean,278181,0.0807288,NaN,NaN,NaN,NaN,0.417052,168798,599026,27108.6,...,0.00812979,0.000595101,0.000507299,0.000334947,0.00640245,0.00700021,0.0343619,0.267395,0.265474,1.89997
std,102790,0.272419,NaN,NaN,NaN,NaN,0.722121,237123,402491,14493.7,...,0.0897982,0.0243875,0.0225176,0.0182985,0.0838491,0.110757,0.204685,0.916002,0.794056,1.86929
min,100002,0,NaN,NaN,NaN,NaN,0,25650,45000,1615.5,...,0,0,0,0,0,0,0,0,0,0
25%,189146,0,NaN,NaN,NaN,NaN,0,112500,270000,16524,...,0,0,0,0,0,0,0,0,0,0
50%,278202,0,NaN,NaN,NaN,NaN,0,147150,513531,24903,...,0,0,0,0,0,0,0,0,0,1
75%,367142,0,NaN,NaN,NaN,NaN,1,202500,808650,34596,...,0,0,0,0,0,0,0,0,0,3
max,456255,1,NaN,NaN,NaN,NaN,19,1.17e+08,4.05e+06,258026,...,1,1,1,1,4,9,8,27,261,25
counts,307511,307511,307511,307511,307511,307511,307511,307511,307511,307499,...,307511,307511,307511,307511,265992,265992,265992,265992,265992,265992
uniques,307511,2,2,3,2,2,15,2548,5603,13672,...,2,2,2,2,5,9,9,24,11,25


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,48744,NaN,NaN,NaN,NaN,48744,48744,48744,48720,48744,...,48744,48744,48744,48744,42695,42695,42695,42695,42695,42695
mean,277797,NaN,NaN,NaN,NaN,0.397054,178432,516740,29426.2,462619,...,0.00155917,0,0,0,0.00210798,0.00180349,0.00278721,0.00929851,0.546902,1.98377
std,103170,NaN,NaN,NaN,NaN,0.709047,101523,365397,16016.4,336710,...,0.0394559,0,0,0,0.0463727,0.0461324,0.0540374,0.110924,0.693305,1.83887
min,100001,NaN,NaN,NaN,NaN,0,26941.5,45000,2295,45000,...,0,0,0,0,0,0,0,0,0,0
25%,188558,NaN,NaN,NaN,NaN,0,112500,260640,17973,225000,...,0,0,0,0,0,0,0,0,0,0
50%,277549,NaN,NaN,NaN,NaN,0,157500,450000,26199,396000,...,0,0,0,0,0,0,0,0,0,2
75%,367556,NaN,NaN,NaN,NaN,1,225000,675000,37390.5,630000,...,0,0,0,0,0,0,0,0,1,3
max,456250,NaN,NaN,NaN,NaN,20,4.41e+06,2.2455e+06,180576,2.2455e+06,...,1,0,0,0,2,2,2,6,7,17
counts,48744,48744,48744,48744,48744,48744,48744,48744,48720,48744,...,48744,48744,48744,48744,42695,42695,42695,42695,42695,42695
uniques,48744,2,2,2,2,11,606,2937,7491,677,...,2,1,1,1,3,3,3,7,8,16


In [9]:
train, test = tables

In [10]:
len(train), len(test)

(307511, 48744)

In [71]:
cat_vars = ['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
           'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
           'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
           'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 
           'OCCUPATION_TYPE','CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
           'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 
           'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
           'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'HOUSETYPE_MODE', 
            'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 
            'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
            'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
            'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 
            'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
            'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_YEAR']

contin_vars = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
              'EXT_SOURCE_3', 'YEARS_BEGINEXPLUATATION_AVG', 'APARTMENTS_AVG', 'COMMONAREA_AVG', 'YEARS_BEGINEXPLUATATION_MODE',
               'APARTMENTS_MODE', 'COMMONAREA_MODE', 'YEARS_BEGINEXPLUATATION_MEDI', 'APARTMENTS_MEDI', 'COMMONAREA_MEDI',
               'TOTALAREA_MODE']
n = len(train); n

307511

In [64]:
dep = 'TARGET'
joined = train[cat_vars+contin_vars+[dep]].copy()

In [66]:
test[dep] = 0
joined_test = test[cat_vars+contin_vars+[dep]].copy()

In [67]:
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

In [68]:
apply_cats(test, joined)

In [69]:
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')

In [72]:
idxs = get_cv_idxs(n, val_pct=150000/n)
joined_samp = joined.iloc[idxs].set_index("SK_ID_CURR")
samp_size = len(joined_samp); samp_size

150000

In [73]:
samp_size = n
joined_samp = joined.set_index("SK_ID_CURR")

In [74]:
joined_samp.head(2)

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,APARTMENTS_AVG,COMMONAREA_AVG,YEARS_BEGINEXPLUATATION_MODE,APARTMENTS_MODE,COMMONAREA_MODE,YEARS_BEGINEXPLUATATION_MEDI,APARTMENTS_MEDI,COMMONAREA_MEDI,TOTALAREA_MODE,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,Cash loans,M,N,Y,0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,...,0.0247,0.0143,0.9722,0.0252,0.0144,0.9722,0.0250,0.0144,0.0149,1
100003,Cash loans,F,N,N,0,Family,State servant,Higher education,Married,House / apartment,...,0.0959,0.0605,0.9851,0.0924,0.0497,0.9851,0.0968,0.0608,0.0714,0


In [75]:
df, y, nas, mapper = proc_df(joined_samp, dep, do_scale=True)
yl = np.log(y)

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [76]:
joined_test = joined_test.set_index("SK_ID_CURR")

In [78]:
df_test, _, nas, mapper = proc_df(joined_test, dep, do_scale=True,
                                  mapper=mapper, na_dict=nas)

In [79]:
df.head(2)

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,YEARS_BEGINEXPLUATATION_AVG,APARTMENTS_AVG,COMMONAREA_AVG,YEARS_BEGINEXPLUATATION_MODE,APARTMENTS_MODE,COMMONAREA_MODE,YEARS_BEGINEXPLUATATION_MEDI,APARTMENTS_MEDI,COMMONAREA_MEDI,TOTALAREA_MODE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,2,1,2,1,7,8,5,4,2,...,0.960978,-0.345180,0.018432,0.961653,-0.327407,0.034883,0.960861,-0.342056,0.020727,-0.411375
100003,1,1,1,1,1,2,5,2,2,2,...,0.987275,0.396431,1.012286,0.987949,0.380977,0.814370,0.987155,0.401242,1.017847,0.197887


In [90]:
#df[df.SK_ID_CURR == 230633]
#df.iloc[230633]
df.columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL',
       'FLAG_EMP_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION',
       'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'HOUSETYPE_MODE',
       'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOC

In [86]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

val_idx

[230633,
 230634,
 230635,
 230636,
 230637,
 230638,
 230639,
 230640,
 230641,
 230642,
 230643,
 230644,
 230645,
 230646,
 230647,
 230648,
 230649,
 230650,
 230651,
 230652,
 230653,
 230654,
 230655,
 230656,
 230657,
 230658,
 230659,
 230660,
 230661,
 230662,
 230663,
 230664,
 230665,
 230666,
 230667,
 230668,
 230669,
 230670,
 230671,
 230672,
 230673,
 230674,
 230675,
 230676,
 230677,
 230678,
 230679,
 230680,
 230681,
 230682,
 230683,
 230684,
 230685,
 230686,
 230687,
 230688,
 230689,
 230690,
 230691,
 230692,
 230693,
 230694,
 230695,
 230696,
 230697,
 230698,
 230699,
 230700,
 230701,
 230702,
 230703,
 230704,
 230705,
 230706,
 230707,
 230708,
 230709,
 230710,
 230711,
 230712,
 230713,
 230714,
 230715,
 230716,
 230717,
 230718,
 230719,
 230720,
 230721,
 230722,
 230723,
 230724,
 230725,
 230726,
 230727,
 230728,
 230729,
 230730,
 230731,
 230732,
 230733,
 230734,
 230735,
 230736,
 230737,
 230738,
 230739,
 230740,
 230741,
 230742,
 230743,
 

In [82]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    fpr, tpr, thresholds = metrics.roc_curve(targ, y_pred)
    return thresholds

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

In [89]:
??ColumnarModelData.from_data_frame

In [83]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test)

KeyError: "['SK_ID_CURR'] not in index"